In [60]:
import csv
urls_delfi = []

with open('delfi_päevauudised_2020_03122020.csv', 'r', encoding = 'utf-8') as f:
    reader = csv.reader(f)
    for line in reader:
        if line != []:
            urls_delfi.append(line[0])

In [62]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

headers = requests.utils.default_headers()
headers.update(
  { 'User-Agent': 'my crawler (anneliis@ut.ee) / on studying purpose', }
)
pealkiri = soup.find("meta", {"property" :"og:title"})['content']
artiklid_delfi_politicsf = {}
pealkirjad_delfi_politics = []
tekstid_delfi_politics = []
väljaanded_delfi_politics = []
ajad_delfi_politics =[]
thearticle = []

for url in tqdm(urls_delfi):
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    yhendatud_tekst = []
    if soup.find("meta", {"property" :"og:title"})['content']:
        pealkiri = soup.find("meta", {"property" :"og:title"})['content']
        pealkirjad_delfi_politics.append(pealkiri)

    #<div class="article__body" itemprop="articleBody">
    tekst = soup.findAll("div", {"class" : "article__body"})#.get_text(strip=True)
    #yhendatud_tekst = ' '
    for t in tekst:
        p = t.find_all('p')
        for r in p:
            #yhendatud_tekst += r.text
            yhendatud_tekst.append(r.text)
            #print(r.text)
    tekstid_delfi_politics.append(yhendatud_tekst)
    #tekstid_delfi_politics = " ".join(yhendatud_tekst)
    
    #<meta property="og:site_name" content="Forte" />
    väljaanne = soup.find("meta", {"property" :"og:site_name"})['content']
    väljaanded_delfi_politics.append(väljaanne)
   
    #<div class="article__date">12.11.2020, 09:28</div>
    if soup.find("meta", {"property" : "article:published_time"})['content']:#.get_text(strip=True):
        aeg = soup.find("meta", {"property" : "article:published_time"})['content']
        ajad_delfi_politics.append(aeg)
artikkel = [' '.join(article) for article in tekstid_delfi_politics]
artiklid_delfi_politics = {'title': pealkirjad_delfi_politics, 'text': artikkel,'subject': väljaanded_delfi_politics, 'date': ajad_delfi_politics}


100%|████████████████████████████████████████████████████████████████████████████| 6226/6226 [1:21:29<00:00,  1.27it/s]


In [64]:
data_delfi = pd.DataFrame(artiklid_delfi_politics, columns = ['title', 'text','subject', 'date'])

In [66]:
data_delfi.to_csv('data_delfi_arhiiv_03122020.csv',index=False)